In [1]:
NAME = 'Jason Ehlenberger'
COLLABORATORS = 'N/A'

In [2]:
import os
import zipfile as zp
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterstats import zonal_stats

In [3]:
# Path to the directory
directory = 'Data\Assignment2'

# Check if the directory exists
if os.path.isdir(directory):
    src_dir = directory
else:
    src_dir = None

print(src_dir)

Data\Assignment2


In [4]:
# scr_dir = 'https://jupyter.geo.appstate.edu/user/ehlenbergerjl/lab/tree/Module6_assignment2/'

In [6]:
def data_path(directory, filename):
    file_path = os.path.join(directory, filename)
    return file_path

In [8]:
# Use the new function to set the path
shpfn = data_path(src_dir, 'counties_subset_Albers4.shp')
data = gpd.read_file(shpfn)
data.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,20,001,00484970,0500000US20001,20001,Allen,06,1295763786,13022381,"POLYGON ((41438.116 1634609.881, 41463.125 163..."
1,20,037,00484988,0500000US20037,20037,Crawford,06,1527498199,13574680,"POLYGON ((79758.943 1613697.800, 79759.066 161..."
2,20,125,00485027,0500000US20125,20125,Montgomery,06,1666900068,20430024,"POLYGON ((3120.618 1558329.024, 3108.760 15615..."
3,20,021,00484980,0500000US20021,20021,Cherokee,06,1521791027,8951473,"POLYGON ((81161.907 1589243.382, 88184.451 158..."
4,20,205,00485063,0500000US20205,20205,Wilson,06,1477382268,12052249,"POLYGON ((3279.132 1597571.957, 3316.386 15982..."


In [9]:
rstfn = data_path(src_dir,'crop_cover_2019.tif')
raster = rasterio.open(rstfn)

type(raster)

rasterio.io.DatasetReader

In [10]:
raster.crs

In [11]:
with rasterio.open(data_path(src_dir, 'crop_cover_2019.tif')) as raster_crop:
    crop_val = raster_crop.read()
crop_val
type(raster)

rasterio.io.DatasetReader

In [12]:
raster.bounds

BoundingBox(left=-30012.21913293552, bottom=1544807.0280387579, right=154937.7808670645, top=1673717.0280387579)

In [13]:
# Raster units & Resolution
print(f'The raster units are {raster.crs.linear_units}s and the resolution is {raster.res}.')

AttributeError: 'NoneType' object has no attribute 'linear_units'

In [ ]:
dst_crs = 'EPSG:9712'

In [ ]:
# step 1: Calculate transform array and shape of reprojected raster 
with rasterio.open(data_path(src_dir, 'crop_cover_2019.tif')) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)

In [ ]:
    # step 2: We copy metadata from source dataset to dictionary object `kwargs`
    kwargs = src.meta.copy()
    # step 3: we update information in kwargs object with destination crs 
    kwargs.update({
    'crs': dst_crs,
    'transform': transform, 
    'width': width, 
    'height': height
    })

In [ ]:
 # step 4: for each band (i), we reproject the calculated parameters froms source to destination      
    with rasterio.open(data_path(src_dir,'crop_2018_subset2_pr.tif', 'w', **kwargs)) as dst:
        for i in range(1, src.count +1):
            reproject(
                source = rasterio.band(src, i), 
                destination = rasterio.band(dst, i), 
                src_transform = src.transform, 
                src_crs = src.crs, 
                dst_transform = transform, 
                dst_crs = dst_crs, 
                resampling = Resampling.nearest)
    # make sure to close the raster file
        dst.close()